In [39]:
# using ChromeDriver 89

from selenium import webdriver
from selenium.webdriver.common.keys import Keys # used to send keys

# https://selenium-python.readthedocs.io/waits.html
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


PATH = './ChromeDriver/chromedriver'
driver = webdriver.Chrome(PATH)

In [41]:
# opening a website
driver.get("https://tutti.ch")

# get title
print(driver.title)

Gratis Inserate in deiner Nähe - tutti.ch


In [42]:
# using the search bar
search = driver.find_element_by_name('search')
search.send_keys('test')
search.send_keys(Keys.RETURN)

In [43]:
# make sure page is loaded
try:
    main = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "root"))
    )
    # find all titles by xpath search
    test = driver.find_elements_by_xpath('//a[@data-automation="ad-subject"]')
    for title in test:
        print(title.text)
except:
    driver.quit()


Top Wii Games (Einzelverkauf, siehe Beschreibung)
Top PS2 Games (Einzelverkauf, siehe Beschreibung)

Englisch 5./6. Klasse & Übungsblock Englisch 5./6. Klasse
Nintendo Switch Neu
iPod Nano - 1st Generation !!!
Gitarren Ankauf / Tausch / Testen
Maxxus Lifeplate 3.1 Vibrationstrainer mit Funksender
TAMRON SP 24-70mm F/2.8 Di VC USD CANON
STRAUSS MERTENS startet ab sofort seine MEGA Aktion!
E-Bike Wheeler E-Falcon - 29 Zoll Räder
Anki Cozmo Roboter
Alesis ADAT XT 8 + IMG Stage Line 19" Rack Case
Nintendo DS Spiel; Gehirn Jogging
Cyclette ergometro orizzontale
Automatik Batterieladegerät HP
Luftbefeuchter
Brother P-Touch M-K221BZ 9mm schwarz/weiss
Samsung Galaxy Buds
1 Million Parfum - NEU!
15.6 Zoll, Acer Aspire mit Windows 10, Office 2019, SSD
Robuste Hundebox in gutem Zustand
VMF AGM Deep Cycle EV Batterie 12 V 63 Ah EV12-63 test websh
JBL Harman InEar Kopfhörer Wireless Reflect Flow
HEAD WORLDCUP REBELS i GSR SLR 155 cm; TEST 2020 Ski
HEAD WORLDCUP REBELS i RACE 165 cm; TEST 2020 Ski
H

In [9]:
# close the browser
driver.quit()